In [1]:
import hashlib, os, sqlite3, numpy as np
from datetime import datetime, timedelta
import pandas as pd
from funcoes.func import *

2024-02-17 17:28:44.722 WARNING streamlit.runtime.caching.cache_data_api: No runtime found, using MemoryCacheStorageManager
2024-02-17 17:28:44.723 WARNING streamlit.runtime.caching.cache_data_api: No runtime found, using MemoryCacheStorageManager
2024-02-17 17:28:44.724 WARNING streamlit.runtime.caching.cache_data_api: No runtime found, using MemoryCacheStorageManager
2024-02-17 17:28:44.725 WARNING streamlit.runtime.caching.cache_data_api: No runtime found, using MemoryCacheStorageManager


In [23]:
expires_at = datetime.now() + timedelta(hours=0.3)
expires = expires_at.isoformat()
expires

'2024-02-17T10:01:36.485260'

In [88]:
id = 50

con = sqlite3.connect('db/db_main_app.db')
cur = con.cursor()
cur.execute(f"""
        SELECT *, timediff(data_liberacao, data_chegada) as diff2 FROM tb_movimento WHERE id_carga = {id}
    """)
res = cur.fetchone()

if res[0] != None:
    print('sem registro. pode excluir')
else:
    print(res[0])

res

sem registro. pode excluir


(10,
 50,
 '14/02/2024 12:00:00',
 '14/02/2024 13:42:00',
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None)

In [73]:
def config_df_painel(df):
    df = df.to_frame()
    print(type(df))
    print(df['status'].values[0])
    #data_chegada = df['data_chegada'].values[0]
    #data_liberacao = df['data_liberacao']
    #data_entrada = df['data_entrada']
    #data_saida = df['data_saida']

    #if data_chegada != '':
    #    if data_liberacao == '':
    #        res = 'Aguardando liberação'
    #    elif data_entrada == '':
    #        res = 'Aguardando entrada'
    #    elif data_saida == '':
    #        res = 'Em descarga'
    #    else:
    #        res = 'Desconhecido'
    #else:
    res = 'Desconhecido'

    return df

In [61]:
type(df)

pandas.core.frame.DataFrame

In [136]:
carga = 52

con = sqlite3.connect('db/db_main_app.db')
df = pd.read_sql(f"SELECT *, STRFTIME('%d/%m/%Y %H:%M', data_chegada) as teste FROM tb_movimento where id_carga = {carga}", con=con)
con.close()

In [137]:
df

,id,id_carga,data_chegada,data_liberacao,data_entrada,data_saida,nota_fiscal,peso_bt,peso_liq,nome_motorista,status,teste
0,11,52,14/02/2024 18:00:00,14/02/2024 18:10:00,None,None,None,None,None,None,None,None


In [141]:
df['teste_data_cheg'] = pd.to_datetime(df['data_chegada'])
df['teste_data_lib'] = pd.to_datetime(df['data_liberacao'])
df['dif'] = df['teste_data_lib'].sub(df['teste_data_cheg']).view('int64') // (60*10**9)
df['status_chegada'] = df['dif'].apply(lambda x: 'Atrasado' if x > 20 else 'Adiantado' if x < -20 else 'No horário')
#df['teste_2'] = pd.Timedelta(df['data_chegada'] - df['data_liberacao'], unit='min')

C:\Users\mathe\AppData\Local\Temp\ipykernel_304\2586559424.py:1: UserWarning: Parsing dates in %d/%m/%Y %H:%M:%S format when dayfirst=False (the default) was specified. Pass `dayfirst=True` or specify a format to silence this warning.
  df['teste_data_cheg'] = pd.to_datetime(df['data_chegada'])
C:\Users\mathe\AppData\Local\Temp\ipykernel_304\2586559424.py:2: UserWarning: Parsing dates in %d/%m/%Y %H:%M:%S format when dayfirst=False (the default) was specified. Pass `dayfirst=True` or specify a format to silence this warning.
  df['teste_data_lib'] = pd.to_datetime(df['data_liberacao'])
C:\Users\mathe\AppData\Local\Temp\ipykernel_304\2586559424.py:3: FutureWarning: Series.view is deprecated and will be removed in a future version. Use ``astype`` as an alternative to change the dtype.
  df['dif'] = df['teste_data_lib'].sub(df['teste_data_cheg']).view('int64') // (60*10**9)


In [183]:
df

,id_carga,data_chegada,data_liberacao,data_entrada,data_saida,nota_fiscal,peso_bt,peso_liq,nome_motorista,status,...,transportador,data_criacao,data_agendamento,data_entrega,tipo_veiculo,tipo_operacao,fornecedor,status_operacao,dif_chegada,status_chegada
0,43,2024-02-14 12:00:00,NaT,NaT,NaT,None,None,None,None,None,...,Newsea Logistics,2024-02-12 15:58:47,2024-02-12 12:00:00,13/02/2024 02:00:00,Carreta 24T,Saída,Planta_SP_2,Aguardando liberação,-2880,Atrasado
1,50,2024-02-14 12:00:00,2024-02-14 13:00:00,NaT,NaT,None,None,None,None,None,...,Nippon Express,2024-02-13 11:21:23,2024-02-14 13:00:00,,Truck 14T,Entrada,GRD Transportes,Aguardando entrada,60,Adiantado
2,52,2024-02-14 18:00:00,2024-02-14 18:10:00,NaT,NaT,None,None,None,None,None,...,GRD Transportes,2024-02-14 18:39:02,2024-02-15 12:00:00,19/02/2024 11:00:00,Carreta 24T,Saída,Planta_SP_1,Aguardando entrada,1080,Adiantado
3,53,2024-02-14 12:00:00,2024-02-14 13:00:00,NaT,NaT,None,None,None,None,None,...,GRD Transportes,2024-02-14 18:39:06,2024-02-14 12:00:00,19/02/2024 11:00:00,Carreta 24T,Saída,Planta_SP_1,Aguardando entrada,0,No horário


In [2]:
con = sqlite3.connect('db/db_main_app.db')
df = pd.read_sql("""
            SELECT
                *,
                IIF(data_chegada IS NULL, "Aguardando chegada",
                    IIF(data_liberacao IS NULL, "Aguardando liberação",
                        IIF(data_entrada IS NULL, "Aguardando entrada",
                            IIF(data_saida IS NULL, "Em descarga", "Desconhecido")
                        )
                    )
                )	as status_operacao
            FROM
                tb_movimento t1
            LEFT JOIN
                tb_carga t2
            ON
                t1.id_carga = t2.id
            WHERE
                data_chegada IS NOT NULL AND data_saida IS NULL
        """, con=con)

df = df.drop('id', axis='columns')

df['data_chegada'] = pd.to_datetime(df['data_chegada'], dayfirst=True)
df['data_liberacao'] = pd.to_datetime(df['data_liberacao'], dayfirst=True)
df['data_entrada'] = pd.to_datetime(df['data_entrada'], dayfirst=True)
df['data_saida'] = pd.to_datetime(df['data_saida'], dayfirst=True)
df['data_agendamento'] = pd.to_datetime(df['data_agendamento'], dayfirst=True)
df['data_criacao'] = pd.to_datetime(df['data_criacao'], dayfirst=True)
df['data_entrada'] = pd.to_datetime(df['data_entrada'], dayfirst=True)


df['dif_chegada'] = df['data_agendamento'].sub(df['data_chegada']).astype('int64') // (60*10**9)
df['status_chegada'] = df['dif_chegada'].apply(lambda x: 'Atrasado' if x < -20 else 'Adiantado' if x > 20 else 'No horário')

In [3]:
df

,id_carga,data_chegada,data_liberacao,data_entrada,data_saida,nota_fiscal,peso_bt,peso_liq,nome_motorista,status,...,transportador,data_criacao,data_agendamento,data_entrega,tipo_veiculo,tipo_operacao,fornecedor,status_operacao,dif_chegada,status_chegada
0,43,2024-02-14 12:00:00,NaT,NaT,NaT,None,None,None,None,None,...,Newsea Logistics,2024-02-12 15:58:47,2024-02-12 12:00:00,13/02/2024 02:00:00,Carreta 24T,Saída,Planta_SP_2,Aguardando liberação,-2880,Atrasado
1,50,2024-02-14 12:00:00,2024-02-14 13:00:00,NaT,NaT,None,None,None,None,None,...,Nippon Express,2024-02-13 11:21:23,2024-02-14 13:00:00,,Truck 14T,Entrada,GRD Transportes,Aguardando entrada,60,Adiantado
2,52,2024-02-14 18:00:00,2024-02-14 18:10:00,NaT,NaT,None,None,None,None,None,...,GRD Transportes,2024-02-14 18:39:02,2024-02-15 12:00:00,19/02/2024 11:00:00,Carreta 24T,Saída,Planta_SP_1,Aguardando entrada,1080,Adiantado
3,53,2024-02-14 12:00:00,2024-02-14 13:00:00,NaT,NaT,None,None,None,None,None,...,GRD Transportes,2024-02-14 18:39:06,2024-02-14 12:00:00,19/02/2024 11:00:00,Carreta 24T,Saída,Planta_SP_1,Aguardando entrada,0,No horário


In [10]:
import requests

url = "https://api.tomtom.com/routing/matrix/2?key=bAvp2Uy2frLuz9BunjnGmwDErsRLPZcB"

response = requests.post(url)

print(response.json())

JSONDecodeError: Expecting value: line 1 column 1 (char 0)

In [4]:
df = pd.read_sql("""
            SELECT
                *,
                IIF(data_chegada IS NULL, "Aguardando chegada",
                    IIF(data_liberacao IS NULL, "Aguardando liberação",
                        IIF(data_entrada IS NULL, "Aguardando entrada",
                            IIF(data_saida IS NULL, "Em descarga", "Desconhecido")
                        )
                    )
                )	as status_operacao
            FROM
                tb_movimento t1
            LEFT JOIN
                tb_carga t2
            ON
                t1.id_carga = t2.id
            WHERE
               data_saida IS NULL
        """, con=con)

df = df.drop('id', axis='columns')

df['data_chegada'] = pd.to_datetime(df['data_chegada'], dayfirst=True)
df['data_liberacao'] = pd.to_datetime(df['data_liberacao'], dayfirst=True)
df['data_entrada'] = pd.to_datetime(df['data_entrada'], dayfirst=True)
df['data_saida'] = pd.to_datetime(df['data_saida'], dayfirst=True)
df['data_agendamento'] = pd.to_datetime(df['data_agendamento'], dayfirst=True)
df['data_criacao'] = pd.to_datetime(df['data_criacao'], dayfirst=True)
df['data_entrada'] = pd.to_datetime(df['data_entrada'], dayfirst=True)


df['dif_chegada'] = df['data_agendamento'].sub(df['data_chegada']).astype('int64') // (60*10**9)
df['status_chegada'] = df['dif_chegada'].apply(lambda x: 'Atrasado' if x < -20 else 'Adiantado' if x > 20 else 'No horário')

In [5]:
df

,id_carga,data_chegada,data_liberacao,data_entrada,data_saida,nota_fiscal,peso_bt,peso_liq,nome_motorista,status,...,transportador,data_criacao,data_agendamento,data_entrega,tipo_veiculo,tipo_operacao,fornecedor,status_operacao,dif_chegada,status_chegada
0,43,2024-02-14 12:00:00,NaT,NaT,NaT,None,None,None,None,None,...,Newsea Logistics,2024-02-12 15:58:47,2024-02-12 12:00:00,13/02/2024 02:00:00,Carreta 24T,Saída,Planta_SP_2,Aguardando liberação,-2880,Atrasado
1,47,NaT,NaT,NaT,NaT,None,None,None,None,None,...,Trans Brasil,2024-02-12 21:07:13,2024-02-14 08:00:00,,Carreta 24T,Entrada,Newsea Logistics,Aguardando chegada,-153722868,Atrasado
2,50,2024-02-14 12:00:00,2024-02-14 13:00:00,NaT,NaT,None,None,None,None,None,...,Nippon Express,2024-02-13 11:21:23,2024-02-14 13:00:00,,Truck 14T,Entrada,GRD Transportes,Aguardando entrada,60,Adiantado
3,52,2024-02-14 18:00:00,2024-02-14 18:10:00,NaT,NaT,None,None,None,None,None,...,GRD Transportes,2024-02-14 18:39:02,2024-02-15 12:00:00,19/02/2024 11:00:00,Carreta 24T,Saída,Planta_SP_1,Aguardando entrada,1080,Adiantado
4,53,2024-02-14 12:00:00,2024-02-14 13:00:00,NaT,NaT,None,None,None,None,None,...,GRD Transportes,2024-02-14 18:39:06,2024-02-14 12:00:00,19/02/2024 11:00:00,Carreta 24T,Saída,Planta_SP_1,Aguardando entrada,0,No horário
5,54,NaT,NaT,NaT,NaT,None,None,None,None,None,...,Trans Brasil,2024-02-16 19:16:51,2024-02-17 09:00:00,,Carreta 24T,Entrada,Trans Brasil,Aguardando chegada,-153722868,Atrasado
6,55,NaT,NaT,NaT,NaT,None,None,None,None,None,...,WFC do Brasil Transportes,2024-02-17 11:12:44,2024-02-13 02:00:00,17/02/2024 06:00:00,Truck 14T,Saída,Planta_SP_1,Aguardando chegada,-153722868,Atrasado
7,56,NaT,NaT,NaT,NaT,None,None,None,None,None,...,WFC do Brasil Transportes,2024-02-17 11:12:52,2024-02-13 02:00:00,15/02/2024 06:00:00,Truck 14T,Saída,Planta_RJ_1,Aguardando chegada,-153722868,Atrasado
8,57,NaT,NaT,NaT,NaT,None,None,None,None,None,...,GRD Transportes,2024-02-17 11:13:09,2024-02-11 00:00:00,14/02/2024 06:00:00,Carreta 24T,Saída,Planta_SP_1,Aguardando chegada,-153722868,Atrasado
9,58,NaT,NaT,NaT,NaT,None,None,None,None,None,...,GRD Transportes,2024-02-17 11:13:16,2024-02-14 06:00:00,,Carreta 24T,Entrada,Newsea Logistics,Aguardando chegada,-153722868,Atrasado


In [40]:
options = list(df.id_carga.unique())
print(options)
options.insert(0,'')
#a = np.insert(options, 1, ' ')
options

[51, 44, 46, 49, 50]


['', 51, 44, 46, 49, 50]

In [44]:
df.data_chegada.str[:10]

0    14/02/202
1    14/02/202
2    14/02/202
3    14/02/202
4    14/02/202
0    14/02/202
Name: data_chegada, dtype: object

In [9]:
dic = {}

dic['nome'] = 'dimas'
dic['sobrenome'] = 'leite'

dic

{'nome': 'matheus', 'sobrenome': 'leite'}

# Cadastrar novo usuário

In [185]:
def hash_senha(senha, salt):
    senha_salt = senha + salt
    hashed_senha = hashlib.sha256(senha_salt.encode('utf-8')).hexdigest()
    return hashed_senha

def cadastrar_usuario(nome,usuario,senha,salt):

    # Conexão com o banco de dados
    print('conecta db - checa se usuario ja existe')
    con = sqlite3.connect('db/db_main_app.db')
    cur = con.cursor()
    cur.execute('select usuario from tb_usuarios where usuario = ?', (usuario,))
    res = cur.fetchone()
    cur.close()

    if res is not None:
        return 'usuário já existe!'
    else:
        print('se usuario nao existe, cria um novo')

        print('Chama hash func')
        senha_hashed = hash_senha(senha,str(salt))
        data_inclusao = datetime.now()
        data_inclusao = data_inclusao.strftime('%d/%m/%Y %H:%M:%S')

        cur = con.cursor()
        cur.execute('insert into tb_usuarios (nome, usuario, senha, salt, logado, data_inclusao) values (?, ?, ?, ?, ?, ?)', (nome, usuario, senha_hashed, str(salt), 0, data_inclusao))
        con.commit()
        con.close()
        return 'usuario cadastrado com sucesso!'

In [186]:
nome = 'Dimas Junior'
usuario = 'djunior'
senha = 'dimas123'
salt = os.urandom(32)

cadastrar_usuario(nome,usuario,senha,salt)

conecta db - checa se usuario ja existe
se usuario nao existe, cria um novo
Chama hash func


'usuario cadastrado com sucesso!'

In [184]:
usuario = 'dimas'
senha = 'dimas123'

# Conexão com o banco de dados
con = sqlite3.connect('db/db_main_app.db')
cur = con.cursor()
cur.execute('select senha, salt from tb_usuarios where usuario = ?', (usuario,))
res = cur.fetchone()
cur.close()
con.close()

if not res:
    print('Usuário ou senha inválidos')
else:
    senha_db = res[0]
    salt_db = res[1]

    senha_salt = senha + salt_db
    hashed_senha = hashlib.sha256(senha_salt.encode('utf-8')).hexdigest()

    if hashed_senha == senha_db:
        print('Conectado')
    else:
        print('Usuário ou senha inválidos')


Usuário ou senha inválidos


In [ ]:
con = sqlite3.connect('db/db_main_app.db')
cur = con.cursor()
cur.execute('select nome_fantasia from tb_cliente')
res = cur.fetchall()
cur.close()
res

lst = []

i = 1
for cliente in res:
    lst.append(cliente[0])

In [ ]:
res[1][0]

In [ ]:
lst

In [ ]:
nome = 'Sarah Weber'
usuario = 'sweber'
senha = '1234'
salt = os.urandom(32)

cadastrar_usuario(nome,usuario,senha,str(salt))

In [ ]:
import yaml, os, pandas as pd

In [ ]:
with open('config/tipo_operacao.yml', 'r') as file:
    dados = yaml.safe_load_all(file)
    lst = []
    for d in dados:
        lst.append(d)

lst_ = []

for i in lst[0]:
    x = i.encode('latin1')
    x = x.decode()
    lst_.append(x)

In [ ]:
lst_

In [ ]:
d[0]

In [ ]:
lst[0][1]

In [ ]:
s = 'DevoluÃ§Ã£o'

x = s.encode('latin1')
x.decode()

In [ ]:
veiculos = {}

with open('config/dados_veiculos.yml', 'r') as file:
    dados = yaml.safe_load_all(file)

    i = 1
    for d in dados:
        print(d)
        veiculos[i] = d['id'], d['tipo'], d['capacidade'], d['eixo']
        i+=1

In [ ]:
pd.DataFrame.from_dict(veiculos, orient='index', columns=['ID','Tipo','Capacidade (ton)','Eixo'])

In [ ]:
veiculos[1]['tipo']

In [ ]:
now = datetime.now()

data = now.strftime('%d/%m/%Y')

In [ ]:
data

In [ ]:
con = sqlite3.connect('db/db_main_app.db')
cur = con.cursor()
cur.execute('select * from tb_carga')
res = cur.fetchall()
cur.close()
con.close()

In [ ]:
con = sqlite3.connect('db/db_main_app.db')
df = pd.read_sql("SELECT * FROM tb_carga", con=con)
con.close()

In [ ]:
df

In [ ]:
carga = 39
result = df.loc[df.id == carga]
res_cliente = result['cliente'].values[0]
res_fornecedor = result['fornecedor'].values[0]
res_transportador = result['transportador'].values[0]

In [ ]:
print(result.shape[0])

In [ ]:
res_transportador

In [ ]:
result.iat[0,1]

In [ ]:
res_cliente

In [2]:
from datetime import datetime

datetime.now().strftime('%d/%m/%Y %H:%M:%S')

'21/02/2024 18:23:59'